In [ ]:
import pandas as pd
import plotly.express as px
import math

In [ ]:
order_report = pd.read_excel("/content/Company X - Order Report.xlsx")
pincode = pd.read_excel(r"./Company X - Pincode Zones.xlsx")
sku = pd.read_excel(r"./Company X - SKU Master.xlsx")
courier_invoice = pd.read_excel("/content/Courier Company - Invoice.xlsx")
rates = pd.read_excel(r"./Courier Company - Rates.xlsx")
result0 = pd.read_excel(r"./Expected_Result.xlsx", sheet_name=0)
result1 = pd.read_excel(r"./Expected_Result.xlsx", sheet_name=1)

In [ ]:
order_report.shape

(400, 3)

In [ ]:
pincode.shape

(124, 3)

In [ ]:
sku.shape

(66, 2)

In [ ]:
courier_invoice.shape

(124, 8)

In [ ]:
rates.shape

(5, 6)

In [ ]:
order_report.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [ ]:
pincode.head()

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [ ]:
sku.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [ ]:
courier_invoice.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [ ]:
rates

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.50,29.5,23.6,13.6,23.6
1,B,1.00,33.0,28.3,20.5,28.3
2,C,1.25,40.1,38.9,31.9,38.9
3,D,1.50,45.4,44.8,41.3,44.8
4,E,2.00,56.6,55.5,50.7,55.5


## check null values

In [ ]:
order_report.isna().sum()

ExternOrderNo    0
SKU              0
Order Qty        0
dtype: int64

In [ ]:
sku.isna().sum()

SKU           0
Weight (g)    0
dtype: int64

In [ ]:
pincode.isna().sum()

Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64

In [ ]:
courier_invoice.isna().sum()

AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

In [ ]:
rates.isna().sum()

Zone                                     0
Weight Slabs                             0
Forward Fixed Charge                     0
Forward Additional Weight Slab Charge    0
RTO Fixed Charge                         0
RTO Additional Weight Slab Charge        0
dtype: int64

#### we are not having any null values in our sample datasets.

## check for duplicated entries

In [ ]:
order_report[order_report.duplicated(keep=False)]           # the duplicated entries can be removed

,ExternOrderNo,SKU,Order Qty
107,2001811363,8904223815859,1.0
110,2001811363,8904223815859,1.0
316,2001806885,8904223819499,2.0
317,2001806885,8904223819499,2.0


In [ ]:
sku[sku.duplicated("SKU",keep=False)]                       # we can remove the duplicated entry here

,SKU,Weight (g)
52,GIFTBOX202002,500
56,GIFTBOX202002,500


In [ ]:
pincode[pincode.duplicated(["Warehouse Pincode", "Customer Pincode"], keep=False)]    # we can remove the duplicated entries here

,Warehouse Pincode,Customer Pincode,Zone
7,121003,140301,b
16,121003,140301,b
43,121003,248001,b
50,121003,248001,b
62,121003,302031,b
63,121003,335001,b
68,121003,302002,b
71,121003,313001,b
72,121003,302002,b
74,121003,302017,b


In [ ]:
courier_invoice[courier_invoice.duplicated(["AWB Code"])]           # no duplicated entries here

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)


In [ ]:
# drop the duplicated entries as discussed above
sku.drop_duplicates(keep="first", inplace=True)
pincode.drop_duplicates(keep="first", inplace=True)
order_report.drop_duplicates(keep="first", inplace=True)

#### we have dropped all the unecessary duplicated entries

## find weight of each orders

### company data

In [ ]:
df1 = pd.merge(order_report, sku, on = "SKU")

In [ ]:
df1.head()

,ExternOrderNo,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001821995,8904223818706,1.0,127
2,2001819252,8904223818706,1.0,127
3,2001816996,8904223818706,1.0,127
4,2001814580,8904223818706,1.0,127


In [ ]:
df1["Weight (g)"] = df1["Weight (g)"]/1000                                  # convert weight in grams -> kilograms
df1["Product Weight (KG)"] = df1["Weight (g)"] * df1["Order Qty"]           # get total product weight

In [ ]:
df1.head()

,ExternOrderNo,SKU,Order Qty,Weight (g),Product Weight (KG)
0,2001827036,8904223818706,1.0,0.127,0.127
1,2001821995,8904223818706,1.0,0.127,0.127
2,2001819252,8904223818706,1.0,0.127,0.127
3,2001816996,8904223818706,1.0,0.127,0.127
4,2001814580,8904223818706,1.0,0.127,0.127


In [ ]:
df1.tail()   # the product weight is calculated properly

,ExternOrderNo,SKU,Order Qty,Weight (g),Product Weight (KG)
393,2001806616,8904223819123,1.0,0.250,0.250
394,2001806567,8904223815804,1.0,0.160,0.160
395,2001806567,8904223818577,1.0,0.150,0.150
396,2001806408,8904223819437,2.0,0.552,1.104
397,2001806232,8904223818645,6.0,0.137,0.822


In [ ]:
# get the total weight of an order by grouping the orders by their ID and sum up
df2 = df1.groupby(["ExternOrderNo"])["Product Weight (KG)"].sum().reset_index()
df2.rename({"Product Weight (KG)":"Order Weight (KG)"}, axis=1, inplace=True)

In [ ]:
df2.head()

,ExternOrderNo,Order Weight (KG)
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245


In [ ]:
pincode.rename({"Zone":"Delivery Zone X"}, axis=1, inplace=True)            # renaming Pincode zone column

In [ ]:
df3 = pd.merge(courier_invoice, pincode, on =["Warehouse Pincode", "Customer Pincode"])

In [ ]:
df3.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Delivery Zone X
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,d
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,d
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,b
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,d


In [ ]:
df4 = df3.loc[:,["Order ID", "Delivery Zone X", "Type of Shipment"]]
df4.rename({"Order ID":"ExternOrderNo"},axis=1, inplace=True)

In [ ]:
df5 = pd.merge(df2,df4, on="ExternOrderNo")

In [ ]:
df5.head()

,ExternOrderNo,Order Weight (KG),Delivery Zone X,Type of Shipment
0,2001806210,0.220,b,Forward charges
1,2001806226,0.480,d,Forward charges
2,2001806229,0.500,d,Forward charges
3,2001806232,1.302,d,Forward charges
4,2001806233,0.245,b,Forward charges


In [ ]:
rates_dict=rates.set_index("Zone").T.to_dict()

In [ ]:
rates_dict

{'A': {'Weight Slabs': 0.5,
  'Forward Fixed Charge': 29.5,
  'Forward Additional Weight Slab Charge': 23.6,
  'RTO Fixed Charge': 13.6,
  'RTO Additional Weight Slab Charge': 23.6},
 'B': {'Weight Slabs': 1.0,
  'Forward Fixed Charge': 33.0,
  'Forward Additional Weight Slab Charge': 28.3,
  'RTO Fixed Charge': 20.5,
  'RTO Additional Weight Slab Charge': 28.3},
 'C': {'Weight Slabs': 1.25,
  'Forward Fixed Charge': 40.1,
  'Forward Additional Weight Slab Charge': 38.9,
  'RTO Fixed Charge': 31.9,
  'RTO Additional Weight Slab Charge': 38.9},
 'D': {'Weight Slabs': 1.5,
  'Forward Fixed Charge': 45.4,
  'Forward Additional Weight Slab Charge': 44.8,
  'RTO Fixed Charge': 41.3,
  'RTO Additional Weight Slab Charge': 44.8},
 'E': {'Weight Slabs': 2.0,
  'Forward Fixed Charge': 56.6,
  'Forward Additional Weight Slab Charge': 55.5,
  'RTO Fixed Charge': 50.7,
  'RTO Additional Weight Slab Charge': 55.5}}

In [ ]:
def get_weight_slab_and_units(data):
    """Description:
            This function gets the company data as input, \
            and allocates weight slabs based on the courier rates dict,\
            We have created the rate_dicts, by indexing Zone in Courier rates dataframe.
            .This also creates the weight slab units for easy price conversion \
            in the next modules.
        data:
            pd.DataFrame
        Returns:
            company_expected_ weight_slabs, company_expected_ weight_slab_units
    """
    exp_weight_slab_units = list()
    exp_weight_slabs = list()
    for i in range(len(data)):
        zone = data.loc[i, "Delivery Zone X"].upper()
        weight = data.loc[i, "Order Weight (KG)"]
        weight_slab_unit = weight / rates_dict[f"{zone}"]["Weight Slabs"]
        exp_weight_slab_unit = math.ceil(weight_slab_unit)
        exp_weight_slab = exp_weight_slab_unit * rates_dict[f"{zone}"]["Weight Slabs"]
        exp_weight_slab_units.append(exp_weight_slab_unit)
        exp_weight_slabs.append(exp_weight_slab)
    return exp_weight_slabs, exp_weight_slab_units

In [ ]:
exp_weight_slabs,exp_weight_slab_units = get_weight_slab_and_units(df5)

In [ ]:
df5["ExpWeightSlabs (KG)"] =  exp_weight_slabs
df5["ExpWeightSlabUnits"] = exp_weight_slab_units

### Charged weights slabs by Courier company

In [ ]:
def get_charged_weight_slab_and_units(data):

    """Description:
            This function gets the courier agency data as input, \
            and allocates weight slabs based on the courier rates dict,\
            We have created the rate_dicts, by indexing Zone in Courier rates dataframe.

        data:
            pd.DataFrame
        Returns:
            Charged weight slabs by Courier agency
    """
    charged_weight_slabs = list()
    for i in range(len(data)):
        zone = data.loc[i, "Zone"].upper()
        weight = data.loc[i, "Charged Weight"]
        weight_slab_unit = weight / rates_dict[f"{zone}"]["Weight Slabs"]
        charged_weight_slab_unit = math.ceil(weight_slab_unit)
        charged_weight_slab = charged_weight_slab_unit * rates_dict[f"{zone}"]["Weight Slabs"]
        charged_weight_slabs.append(charged_weight_slab)
    return charged_weight_slabs

In [ ]:
charged_weight_slabs = get_charged_weight_slab_and_units(courier_invoice)

In [ ]:
courier_invoice["ChargedWeightSlabs (KG)"] = charged_weight_slabs

### Expected Charges by Company X

In [ ]:
def order_forward_charge(order):
    """Description:
            This function gets the company data as input, \
            and allocates forward charges based on rates_dict.\
            We have created the rate_dicts, by indexing Zone in Courier rates dataframe.
            .This function uses the weight slab units for easy price calculations.

        data:
            pd.DataFrame
        Returns:
            Expected Forward charges by the company
    """
    zone = order["Delivery Zone X"].upper()
    weight_unit = order["ExpWeightSlabUnits"]
    if weight_unit==1:                                   # calculate price if order has no additional rates.
        total_rate = rates_dict[f"{zone}"]["Forward Fixed Charge"]
    else:                                                # calculate price if order has additional rates.
        fixed_rate = rates_dict[f"{zone}"]["Forward Fixed Charge"]
        additional_rate = rates_dict[f"{zone}"]["Forward Additional Weight Slab Charge"] * (weight_unit-1)
        total_rate = fixed_rate + additional_rate
    return total_rate

In [ ]:
df5["Forward_charges"] = df5.apply(order_forward_charge, axis=1)

In [ ]:
def order_rto_charge(order):
    """Description:
            This function gets the company data as input, \
            and allocates RTO charges based on rates_dict.\
            We have created the rate_dicts, by indexing Zone in Courier rates dataframe.
            .This function uses the weight slab units for easy price calculations.

        data:
            pd.DataFrame
        Returns:
            Expected RTO charges by the company
    """
    zone = order["Delivery Zone X"].upper()
    weight_unit = order["ExpWeightSlabUnits"]
    shipment = order["Type of Shipment"]
    total_rate=0
    if shipment=="Forward and RTO charges":
        if weight_unit==1:                                   # calculate price if order has no additional rates.
            total_rate = rates_dict[f"{zone}"]["RTO Fixed Charge"]
        else:                                                # calculate price if order has additional rates.
            fixed_rate = rates_dict[f"{zone}"]["RTO Fixed Charge"]
            additional_rate = rates_dict[f"{zone}"]["RTO Additional Weight Slab Charge"] * (weight_unit-1)
            total_rate = fixed_rate + additional_rate
    return total_rate

In [ ]:
df5["RTO_charges"] = df5.apply(order_rto_charge,axis=1)

In [ ]:
total_company_charges = df5["Forward_charges"] + df5["RTO_charges"]

In [ ]:
df5["Expected Charge as per X (Rs.)"] = round(total_company_charges,2)

In [ ]:
df5.head()

,ExternOrderNo,Order Weight (KG),Delivery Zone X,Type of Shipment,ExpWeightSlabs (KG),ExpWeightSlabUnits,Forward_charges,RTO_charges,Expected Charge as per X (Rs.)
0,2001806210,0.220,b,Forward charges,1.0,1,33.0,0.0,33.0
1,2001806226,0.480,d,Forward charges,1.5,1,45.4,0.0,45.4
2,2001806229,0.500,d,Forward charges,1.5,1,45.4,0.0,45.4
3,2001806232,1.302,d,Forward charges,1.5,1,45.4,0.0,45.4
4,2001806233,0.245,b,Forward charges,1.0,1,33.0,0.0,33.0


In [ ]:
df5.tail()

,ExternOrderNo,Order Weight (KG),Delivery Zone X,Type of Shipment,ExpWeightSlabs (KG),ExpWeightSlabUnits,Forward_charges,RTO_charges,Expected Charge as per X (Rs.)
119,2001821995,0.477,b,Forward charges,1.0,1,33.0,0.0,33.0
120,2001822466,1.376,b,Forward charges,2.0,2,61.3,0.0,61.3
121,2001823564,0.672,d,Forward and RTO charges,1.5,1,45.4,41.3,86.7
122,2001825261,1.557,d,Forward and RTO charges,3.0,2,90.2,86.1,176.3
123,2001827036,1.676,e,Forward charges,2.0,1,56.6,0.0,56.6


In [ ]:
df6 = df5.drop("Type of Shipment", axis=1)

### rename the columns for final data

In [ ]:
df7 = pd.merge(df6, courier_invoice, left_on = "ExternOrderNo", right_on = "Order ID")

In [ ]:
df7.head()

,ExternOrderNo,Order Weight (KG),Delivery Zone X,ExpWeightSlabs (KG),ExpWeightSlabUnits,Forward_charges,RTO_charges,Expected Charge as per X (Rs.),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ChargedWeightSlabs (KG)
0,2001806210,0.220,b,1.0,1,33.0,0.0,33.0,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,3.0
1,2001806226,0.480,d,1.5,1,45.4,0.0,45.4,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2,1.5
2,2001806229,0.500,d,1.5,1,45.4,0.0,45.4,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2,1.5
3,2001806232,1.302,d,1.5,1,45.4,0.0,45.4,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.5
4,2001806233,0.245,b,1.0,1,33.0,0.0,33.0,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3,1.0


In [ ]:
df7.rename({"Charged Weight":"Total weight as per Courier Company (KG)",
            "Billing Amount (Rs.)":"Charges Billed by Courier Company (Rs.)",
            "Zone":"Delivery Zone charged by Courier Company",
            "AWB Code":"AWB Number"}, axis=1, inplace=True)

In [ ]:
# calculate price difference between comapny and courier service.
df7["Difference Between Expected Charges and Billed Charges (Rs.)"] =\
                        df7["Expected Charge as per X (Rs.)"] - df7["Charges Billed by Courier Company (Rs.)"]

### upper case delivery zones

In [ ]:
def upper_case(zone):
    return zone.upper()

In [ ]:
df7["Delivery Zone charged by Courier Company"] = df7["Delivery Zone charged by Courier Company"].apply(upper_case)
df7["Delivery Zone X"] = df7["Delivery Zone X"].apply(upper_case)

In [ ]:
rename_columns = {"Delivery Zone X":"Delivery Zone as per X",
 "Order Weight (KG)":"Total weight as per X (KG)",
 "ExpWeightSlabs (KG)":"Weight slab as per X (KG)",
"ChargedWeightSlabs (KG)":"Weight slab charged by Courier Company (KG)"}

In [ ]:
df7.rename(rename_columns, axis=1, inplace=True)

In [ ]:
df7.head()

,ExternOrderNo,Total weight as per X (KG),Delivery Zone as per X,Weight slab as per X (KG),ExpWeightSlabUnits,Forward_charges,RTO_charges,Expected Charge as per X (Rs.),AWB Number,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Weight slab charged by Courier Company (KG),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,0.220,B,1.0,1,33.0,0.0,33.0,1091117221940,2001806210,2.92,121003,140604,B,Forward charges,174.5,3.0,-141.5
1,2001806226,0.480,D,1.5,1,45.4,0.0,45.4,1091117222065,2001806226,0.68,121003,723146,D,Forward charges,90.2,1.5,-44.8
2,2001806229,0.500,D,1.5,1,45.4,0.0,45.4,1091117222080,2001806229,0.71,121003,421204,D,Forward charges,90.2,1.5,-44.8
3,2001806232,1.302,D,1.5,1,45.4,0.0,45.4,1091117222124,2001806232,1.30,121003,507101,D,Forward charges,135.0,1.5,-89.6
4,2001806233,0.245,B,1.0,1,33.0,0.0,33.0,1091117222135,2001806233,0.78,121003,263139,B,Forward charges,61.3,1.0,-28.3


### final Calculations

In [ ]:
result = pd.read_excel("/content/Expected_Result.xlsx", sheet_name=1)

In [ ]:
final_columns = [column.strip() for column in result.columns]    # get final column names from sample final Calculations

In [ ]:
final_calculations = df7[final_columns]                     # filter required columns from df4

In [ ]:
final_calculations.head()

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,1.0,2.92,3.0,B,B,33.0,174.5,-141.5
1,2001806226,1091117222065,0.480,1.5,0.68,1.5,D,D,45.4,90.2,-44.8
2,2001806229,1091117222080,0.500,1.5,0.71,1.5,D,D,45.4,90.2,-44.8
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,45.4,135.0,-89.6
4,2001806233,1091117222135,0.245,1.0,0.78,1.0,B,B,33.0,61.3,-28.3


### order Summary

In [ ]:
# classify orders into 3 categories
correct_charged = final_calculations.loc[
                  final_calculations["Difference Between Expected Charges and Billed Charges (Rs.)"]==0]
over_charged = final_calculations.loc[
                  final_calculations["Difference Between Expected Charges and Billed Charges (Rs.)"]<0]
under_charged = final_calculations.loc[
                  final_calculations["Difference Between Expected Charges and Billed Charges (Rs.)"]>0]

In [ ]:
total_correct_charged = {"Count":len(correct_charged),
                         "Amount":round(correct_charged["Charges Billed by Courier Company (Rs.)"].sum(),1)}

total_over_charged = {"Count":len(over_charged),
                         "Amount":round(over_charged["Difference Between Expected Charges and Billed Charges (Rs.)"].sum(),1)}

total_under_charged = {"Count":len(under_charged),
                         "Amount":round(under_charged["Difference Between Expected Charges and Billed Charges (Rs.)"].sum(),1)}

In [ ]:
total_correct_charged

{'Count': 8, 'Amount': 428.6}

In [ ]:
total_under_charged

{'Count': 2, 'Amount': 47.2}

In [ ]:
total_over_charged

{'Count': 114, 'Amount': -7751.2}

In [ ]:
# order summary
summary = pd.DataFrame(data=[total_correct_charged,total_over_charged, total_under_charged],
                         index=["Total Orders - Correctly Charged",
                                "Total Orders - Over Charged", "Total Orders - Under Charged"])

In [ ]:
summary

,Count,Amount
Total Orders - Correctly Charged,8,428.6
Total Orders - Over Charged,114,-7751.2
Total Orders - Under Charged,2,47.2


### export as excel file

In [ ]:
with pd.ExcelWriter("Expected_result.xlsx") as writer:             # using excel writer to dump two sheets in a single file

    summary.to_excel(writer, sheet_name="Summary")                 # Summary sheet

    final_calculations.to_excel(writer, sheet_name="Calculations",
                                index=False)                       # Calculations sheet